In [ ]:
import pandas as pd

In [ ]:
matches = pd.read_csv('/content/merged1.csv')

In [ ]:
matches = matches.iloc[:, :14]

In [ ]:
matches['Date'] = pd.to_datetime(matches['Date'])
matches['Home_Team_Code'] = matches['HomeTeam'].astype('category').cat.codes
matches['Away_Team_Code'] = matches['AwayTeam'].astype('category').cat.codes
matches['avg_home_goals'] = matches.groupby('HomeTeam')['FTHG'].expanding().mean().reset_index(level=0, drop=True)
matches['avg_away_goals'] = matches.groupby('AwayTeam')['FTAG'].expanding().mean().reset_index(level=0, drop=True)
matches['total_goals'] = matches['FTHG'] + matches['FTAG']
matches['Day_code'] = matches["Date"].dt.dayofweek
matches['target'] = (matches['total_goals'] > 2.5).astype(int)


<ipython-input-134-cb6a827c32ae>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  matches['Date'] = pd.to_datetime(matches['Date'])


In [ ]:
matches

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,...,B365H,B365D,B365A,Home_Team_Code,Away_Team_Code,avg_home_goals,avg_away_goals,total_goals,Day_code,target
0,G1,2017-08-19,Asteras Tripolis,Giannina,1.0,2.0,A,14.0,9.0,9.0,...,1.70,3.40,5.50,3,6,1.000000,2.000000,3.0,5,1
1,G1,2017-08-19,Olympiakos,Larisa,4.0,1.0,H,16.0,3.0,10.0,...,1.17,7.00,15.00,14,11,4.000000,1.000000,5.0,5,1
2,G1,2017-08-19,Xanthi,Lamia,0.0,0.0,D,8.0,4.0,4.0,...,1.67,3.30,6.00,22,10,0.000000,0.000000,0.0,5,0
3,G1,2017-08-20,AEK,Panetolikos,2.0,0.0,H,11.0,10.0,5.0,...,1.36,4.50,9.00,0,17,2.000000,0.000000,2.0,6,0
4,G1,2017-08-20,Kerkyra,Panionios,0.0,1.0,A,4.0,10.0,1.0,...,3.75,3.10,2.10,8,18,0.000000,1.000000,1.0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,G1,2025-02-22,Volos NFC,Levadeiakos,1.0,2.0,A,8.0,10.0,1.0,...,2.35,2.90,3.40,21,12,1.104167,0.627119,3.0,5,1
1844,G1,2025-02-23,Aris,Panetolikos,2.0,1.0,H,23.0,7.0,10.0,...,1.55,3.90,6.00,2,17,1.538462,0.872000,3.0,6,1
1845,G1,2025-02-23,Lamia,Panathinaikos,3.0,1.0,H,8.0,21.0,4.0,...,10.00,4.75,1.33,10,16,0.856000,1.121212,4.0,6,1
1846,G1,2025-02-23,Asteras Tripolis,AEK,0.0,3.0,A,2.0,15.0,0.0,...,7.00,3.90,1.50,3,0,1.226562,1.522727,3.0,6,1


In [ ]:
grouped_matches = matches.groupby("HomeTeam")

In [ ]:
def rolling_averages(group, cols, new_cols):
  group = group.sort_values("Date")
  rolling_stats = group[cols].rolling(3, closed='left').mean()
  group[new_cols]= rolling_stats
  group = group.dropna(subset=new_cols)
  return group

In [ ]:
cols = ["HS", "AS", "HST", "AST", "avg_home_goals", "avg_away_goals"]
new_cols = [f"{c}_rolling" for c in cols]

In [ ]:
matches_rolling = matches.groupby("HomeTeam").apply(lambda x: rolling_averages(x, cols, new_cols))

<ipython-input-139-69a8a1bda503>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("HomeTeam").apply(lambda x: rolling_averages(x, cols, new_cols))


In [ ]:
matches_rolling = matches_rolling.droplevel('HomeTeam')

In [ ]:
matches_rolling.index = range(matches_rolling.shape[0])

In [ ]:
matches_rolling

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,...,avg_away_goals,total_goals,Day_code,target,HS_rolling,AS_rolling,HST_rolling,AST_rolling,avg_home_goals_rolling,avg_away_goals_rolling
0,G1,2017-10-23,AEK,Atromitos,0.0,1.0,A,21.0,9.0,10.0,...,1.000000,1.0,0,0,16.000000,5.666667,9.000000,2.333333,2.666667,0.611111
1,G1,2017-11-05,AEK,PAOK,1.0,0.0,H,3.0,4.0,2.0,...,0.200000,1.0,6,0,19.333333,5.333333,10.666667,2.666667,2.750000,0.944444
2,G1,2017-11-27,AEK,Platanias,3.0,0.0,H,27.0,3.0,12.0,...,0.666667,3.0,0,1,13.666667,6.333333,7.666667,3.000000,2.416667,0.844444
3,G1,2017-12-11,AEK,Kerkyra,3.0,1.0,H,11.0,12.0,8.0,...,0.571429,4.0,0,1,17.000000,5.333333,8.000000,2.333333,2.138889,0.622222
4,G1,2017-12-16,AEK,Apollon,0.0,0.0,D,21.0,2.0,7.0,...,0.500000,0.0,5,0,13.666667,6.333333,7.333333,2.666667,2.150794,0.479365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764,G1,2020-02-16,Xanthi,Lamia,0.0,0.0,D,10.0,4.0,3.0,...,0.666667,0.0,6,0,6.333333,7.666667,4.000000,4.666667,1.041688,0.860684
1765,G1,2020-03-01,Xanthi,PAOK,1.0,1.0,D,4.0,11.0,1.0,...,1.720930,2.0,6,0,7.000000,6.666667,3.333333,4.000000,1.032530,0.835043
1766,G1,2020-06-21,Xanthi,Panetolikos,1.0,1.0,D,11.0,3.0,5.0,...,0.777778,2.0,6,0,7.000000,7.333333,2.666667,2.666667,1.031949,1.142019
1767,G1,2020-06-24,Xanthi,Atromitos,1.0,0.0,H,3.0,4.0,2.0,...,1.266667,1.0,2,0,8.333333,6.000000,3.000000,2.333333,1.023264,1.055125


In [ ]:
predictors = ['Home_Team_Code', 'Away_Team_Code', 'Day_code',]

In [ ]:
def make_predictions(data, predictors):
  train = data[data['Date'] < '2024-02-24']
  test = data[data['Date'] > '2024-02-24']
  rf.fit(train[predictors], train['target'])
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test['target'], predicted=preds), index=test.index)
  error = precision_score(test['target'], preds)
  return combined, error

In [ ]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [ ]:
precision

0.6266666666666667

In [ ]:
combined = combined.merge(matches_rolling[["Date", "HomeTeam", "AwayTeam", "total_goals"]], left_index=True, right_index=True)

In [ ]:
combined

,actual,predicted,Date,HomeTeam,AwayTeam,total_goals
111,1,1,2024-02-28,AEK,Giannina,6.0
112,0,1,2024-03-31,AEK,Olympiakos,1.0
113,1,0,2024-04-07,AEK,PAOK,4.0
114,0,1,2024-04-14,AEK,Aris,2.0
115,1,0,2024-04-24,AEK,Panathinaikos,3.0
...,...,...,...,...,...,...
1721,1,0,2024-12-14,Volos NFC,Asteras Tripolis,3.0
1722,1,1,2025-01-05,Volos NFC,AEK,6.0
1723,0,0,2025-01-24,Volos NFC,Atromitos,1.0
1724,0,0,2025-02-08,Volos NFC,Panetolikos,1.0
